In [14]:
!pip install transformers torch datasets
!pip install tqdm
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, pipeline, AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, AutoModelForQuestionAnswering, BertForQuestionAnswering
from datasets import load_dataset, Dataset
import torch
import torch.nn.functional as F
import re
from nltk.tokenize import sent_tokenize
import nltk
import json
import os
import numpy as np
os.environ["WANDB_DISABLED"] = "true"
nltk.download('punkt')
nltk.download('punkt_tab')
from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Tokenize text at sentence level for BERT tokenizer.  This is generic and will run for any relevant path so do it first.


In [2]:
def load_text(file_path):
    """
    Load text from a file.
    """
    print("Loading the text from the file...")
    with open(book_path, 'r', encoding='utf-8') as file:
        return file.read()

def remove_gutenberg_header_footer(text, start_marker, end_marker):
    """
    Remove the header and footer from Project Gutenberg text.

    Finds specified start and end markers and returns the content in between.

    Parameters:
        text (str): The text to clean.
        start_marker (str): Marker indicating the start of main content.
        end_marker (str): Marker indicating the end of main content.

    Returns:
        str: Text with header and footer removed, or original text if markers are not found.
    """
    start_index = text.find(start_marker)
    if start_index == -1:
        return text

    end_index = text.find(end_marker)
    if end_index == -1:
        return text

    cleaned_text = text[start_index + len(start_marker):end_index].strip()
    return cleaned_text

book_path = "/content/drive/My Drive/Cosc524 - Collaboration/a study in scarlet.txt"

# here we are cleaning the text and removing the headers and footers
raw_text = load_text(book_path)

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"

cleaned_text = remove_gutenberg_header_footer(raw_text, start_marker, end_marker)

print("Cleaned text sample:\n")
print(cleaned_text[:500])

def tokenize_sentences(text):
    """
    Tokenize the input text into sentences using NLTK.

    Parameters:
        text (str): The text to tokenize.

    Returns:
        list: A list of tokenized sentences.
    """
    print("Tokenizing the text into sentences...")
    sentences = sent_tokenize(text)
    print(f"Total sentences: {len(sentences)}")
    return sentences

my_sentences = tokenize_sentences(cleaned_text)

Loading the text from the file...
Cleaned text sample:

A STUDY IN SCARLET

By A. Conan Doyle




CONTENTS

 A STUDY IN SCARLET.

 PART I.
 CHAPTER I. MR. SHERLOCK HOLMES.
 CHAPTER II. THE SCIENCE OF DEDUCTION.
 CHAPTER III. THE LAURISTON GARDENS MYSTERY
 CHAPTER IV. WHAT JOHN RANCE HAD TO TELL.
 CHAPTER V. OUR ADVERTISEMENT BRINGS A VISITOR.
 CHAPTER VI. TOBIAS GREGSON SHOWS WHAT HE CAN DO.
 CHAPTER VII. LIGHT IN THE DARKNESS.

 PART II. THE COUNTRY OF THE SAINTS
 CHAPTER I. ON THE GREAT ALKALI PLAIN.
 CHAPTER II. THE FLOWER OF UTAH.
 CHAPTER III. J
Tokenizing the text into sentences...
Total sentences: 2208


In [ ]:
# # For text generation
# generator = pipeline('fill-mask', model='bert-base-uncased')

# # For text retrieval

# #I should try swapping to the question answer BERT model!
# retriever_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# retriever_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None):

#   """Retrieves the most relevant sentences based on cosine similarity.

#   Args:
#     query: The input query string.
#     top_k: The number of top sentences to retrieve.

#   Returns:
#     A list of the top_k most relevant sentences.
#   """
#   query_embedding = retriever_model(**retriever_tokenizer(query, return_tensors="pt")).last_hidden_state[:, 0, :]
#   similarities = F.cosine_similarity(embeddings, query_embedding, dim=1) # Assume 'embeddings' is pre-calculated
#   top_indices = torch.topk(similarities, top_k).indices
#   relevant_sentences = [sentences[idx] for idx in top_indices]  # Assume 'sentences' is pre-calculated
#   return relevant_sentences

In [ ]:
# def generate_text_with_rag(prompt, top_k=5, embeddings=None, sentences=None):
#   """Generates text using RAG with BERT.

#   Args:
#     prompt: The input prompt for text generation.
#     top_k: The number of retrieved sentences to use for augmentation.

#   Returns:
#     The generated text.
#   """
#   relevant_info = retrieve_relevant_info(prompt, top_k=top_k, embeddings=embeddings, sentences=sentences)
#   augmented_prompt = f"{prompt} [SEP] {' '.join(relevant_info)} [MASK]."  # Add [MASK] token
#   generated_text = generator(augmented_prompt)[0]['sequence']
#   return generated_text

Generate Sentence Embeddings with Bert

In [3]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_sentence_embeddings(sentences):
    # Initialize an empty list to accumulate embeddings
    all_embeddings = []

    # Process each sentence one at a time
    for sentence in tqdm(sentences, desc="Items remaining"):
        # Tokenize the sentence
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)

        # Forward pass through the model (no gradients needed)
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (using mean pooling)
        last_hidden_state = outputs.last_hidden_state
        sentence_embedding = last_hidden_state.mean(dim=1)  # Shape: [1, hidden_size]

        # Append the embedding to the list
        all_embeddings.append(sentence_embedding.squeeze())

    # Stack all embeddings into a single tensor [num_sentences, hidden_size]
    all_embeddings_tensor = torch.stack(all_embeddings)

    return all_embeddings_tensor  # Shape: [num_sentences, hidden_size]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Load sentence embeddings or generate your own

In [5]:
# my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_embeddings.pt')
my_embeddings = generate_sentence_embeddings(my_sentences)
# prompt = "Who was the murderer?"
# generated_text = generate_text_with_rag(prompt=prompt, top_k=5, embeddings=my_embeddings, sentences=my_sentences)
# # print(generated_text)
# for sentence in sent_tokenize(generated_text):
#   print(sentence)

<ipython-input-5-c4b2d6961f68>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_e

Working on RAG doot doot, doot doot

In [ ]:
# generator = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')
# generator = pipeline('question-answering', model='bert-base-uncased')
# def generate_text_with_rag(prompt, top_k=5, embeddings=None, sentences=None):
#     relevant_info = retrieve_relevant_info(prompt, top_k=top_k, embeddings=embeddings, sentences=sentences)
#     context = " ".join(relevant_info)  # Combine retrieved sentences as context
#     generated_text = generator(question=prompt, context=context)['answer']
#     return generated_text

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
# Retrieve relevant sentences based on query
def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None, tokenizer=None, model=None):

    # Compute query embedding
    query_input = tokenizer(query, return_tensors="pt", truncation=True)
    with torch.no_grad():
        query_output = model(**query_input)
        query_embedding = query_output.last_hidden_state.mean(dim=1)  # Average over tokens

    # Compute cosine similarities
    similarities = F.cosine_similarity(embeddings, query_embedding, dim=1)

    # Get top-k most similar sentences
    top_indices = torch.topk(similarities, top_k).indices
    relevant_sentences = [sentences[idx] for idx in top_indices]
    return relevant_sentences


Generate RAG context

In [9]:
# For text generation
generator = pipeline('fill-mask', model='bert-base-uncased')

#I should try swapping to the question answer BERT model!
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = model = BertModel.from_pretrained('bert-base-uncased')

#Our generic query will be "Who is the protagonist in [x]" where x is our story.
query = "Who is the protagonist in a Study in Scarlet?"
relevant_sentences = retrieve_relevant_info(query, top_k=10, embeddings=my_embeddings, sentences=my_sentences, tokenizer=tokenizer, model=model)
context = ' '.join(relevant_sentences)
# print("Relevant Sentences:", relevant_sentences)
for sentence in relevant_sentences:
  print(sentence)

#The context looks ok here.
print(len(context))

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

A STUDY IN SCARLET.
A STUDY IN SCARLET

By A. Conan Doyle




CONTENTS

 A STUDY IN SCARLET.
What is the thirteenth rule in the code of the sainted Joseph Smith?
I might not have gone but for you, and so have
missed the finest study I ever came across: a study in scarlet, eh?
It is almost a facsimile.”

“And who do you expect will answer this advertisement.”

“Why, the man in the brown coat—our florid friend with the square toes.
Of contemporary
literature, philosophy and politics he appeared to know next to
nothing.
“What is the use of having brains in our profession.
You
have asked me if I know the name of the assassin.
What’s that little thing of Chopin’s she plays so
magnificently: Tra-la-la-lira-lira-lay.”

Leaning back in the cab, this amateur bloodhound carolled away like a
lark while I meditated upon the many-sidedness of the human mind.
‘Who talks of murdering a mad dog?
892


In [10]:
#truncate context to match BERT context window size
#The model I am using is 512 window size
def truncate_context(question, context, tokenizer, max_length=512):
    """
    Truncates the context to fit within the model's max token limit.

    Args:
        question: The input question.
        context: The input context string.
        tokenizer: The tokenizer used to calculate token length.
        max_length: The maximum allowed length of tokens.

    Returns:
        A truncated context string.
    """
    tokens = tokenizer(question, context, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)

Testing the Fine Tuned Model

In [11]:
fine_tuned_model_path = "/content/drive/My Drive/Cosc524 - Collaboration/qa_model/finetuned"
dataset_path = "/content/drive/My Drive/Cosc524 - Collaboration/dataset/qa_dataset.json"

fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_path)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Initialize tokenizer and QA pipeline
qa_pipeline = pipeline("question-answering", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

query = "Who is the murderer in A Study in Scarlet?"

#If we change the tokenizer in the parameters it generates an incorrect text answer... but atleast it isnt just a comma. ;-;
# context = truncate_context(query, context, fine_tuned_tokenizer, max_length=512)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)




sidedness


Testing the out of box model

In [12]:
# Initialize tokenizer and QA pipeline
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

query = "Who is the Murderer in A Study in Scarlet?"

#If we change the tokenizer in the parameters it generates an incorrect text answer... but atleast it isnt just a comma. ;-;
# context = truncate_context(query, context, tokenizer, max_length=512)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.





our florid friend with the square toes.


Testing the out of box model fine tuned on question answering.

In [13]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

query = "Who is the Murderer in A Study in Scarlet?"

#If we change the tokenizer in the parameters it generates an incorrect text answer... but atleast it isnt just a comma. ;-;
# context = truncate_context(query, context, tokenizer, max_length=512)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).





the man in the brown coat
